# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [359]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk

### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ellascarola/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ellascarola/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.0 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## 0.1 Load and clean text data

In [360]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [363]:
## your code to subset to one press release and take the string
subset = doj[doj["id"]=="17-1204"]
pharma = subset["contents"].iloc[0]
pharma

## to check that it is a string:
#type(pharma)

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [365]:
## Part A:
pharma_processed = [token for token in wordpunct_tokenize(pharma)
                   if token.isalpha()]
#pharma_processed

In [205]:
## Part B:
pharma_pos = pos_tag(pharma_processed)

#pharma_pos

In [367]:
## Part C:
adj_tags = ["JJ", "JJR", "JJS"]
all_adjectives = [tok[0] for tok in pharma_pos
                 if tok[1] in adj_tags]

adj_series = pd.Series(all_adjectives)
adj_counts = adj_series.value_counts()

top_5_adj = adj_counts.head(5).reset_index()
top_5_adj.columns = ["adjective", "count"]
top_5_adj

,adjective,count
0,former,8
1,opioid,5
2,nationwide,4
3,addictive,3
4,other,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [369]:
## Part A:
spacy_pharma = nlp(pharma)
#print(type(spacy_pharma))

In [371]:
## Part B:
law_entities = []
for tok in spacy_pharma.ents:
    if tok.label_ == "LAW" and tok.text not in law_entities: 
        law_entities.append(tok.text)

print(law_entities)

['RICO', 'the Controlled Substances Act']


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

**Part C:**
According to Herman Law White Collar Criminal Defense, a violation of RICO occurs when a person, in connection with an enterprise, engages in a pattern of racketeering activity that can include the distribution of a controlled substance or wire fruad, meaning that this pharmaceutical kickbacks case can be classified under RICO because it relates to the distribution of opiods (a controlled substance) through a company (Insys Therapeutics) in a way that violates anti-kickback law and involves wire fraud.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [373]:
## Part D:
sentence_lengths =[]
for tok in spacy_pharma.ents:
    if tok.label_ == "DATE" and re.search(r"years?", tok.text.lower()) and tok.text not in sentence_lengths:
        sentence_lengths.append(tok.text)
        
print(sentence_lengths)

['last year', '20 years', 'three years', 'five years']


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [375]:
## PART E:
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(pharma) #breaks pharma into sentences
year_sentences = [s for s in sentences if re.search(r"years?", s.lower())]

print("Sentences that mention year(s):\n")
for s in year_sentences:
    print("-", s, "\n")

Sentences that mention year(s):

- "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. 

- This investigation highlights our commitment to defending our mail system from illegal misuse and ensuring public trust in the mail.”“The U.S. Department of Veterans Affairs, Office of Inspector General will continue to aggressively investigate those that attempt to fraudulently impact programs designed to benefit our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss. 

- The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised releas

**Part E:** If convicted after this indictment, the CEO may be facing up to 20 years in prison and up to 3 years of probation for both his RICO and mail/wire fraud charges; he may face up to 5 years in prison and up to 3 years of probation for his violation of Anti-Kickback Law charge.

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [377]:
## PART A:
doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]
doj_subset.shape

(717, 6)

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [379]:
## NEED TO CHECK THIS! Specifically whether there are inefficiencies - I think it ran in about 2 minutes.

## your code here to define function
def func(string):
    ## Remove named entities from each press release:
    spacy_string = nlp(string)
    named_entities = [tok.text for tok in spacy_string.ents]
    if named_entities:
        pattern = "(" + "|".join([re.escape(ent) for ent in named_entities]) + ")" #this groups the named entities from the string together so that we can get rid of them in the next line
        # we need to use re.escape so that our regex pattern isn't thrown off by the presence of special characters in the strings
        cleaned_text = re.sub(pattern, "", string) #any time one of the named entities (given by the pattern) appears in the string, it will be replaced with ""
## I think it would also be possible to do this step via a for loop that loops over each entity in the named_entities
## list and replaces it with "" in the string, but I'm pretty sure it would take a lot longer since you'd have to loop
 ## through the string one time for each named entity rather than looping through it once and checking for any named entity
    
    else:
        cleaned_text = string
    #just a check:
    #print(cleaned_text)
    
    ## Score the sentiment of the entire press release using SentimentIntensityAnalyzer and polarity_scores
    sia = SentimentIntensityAnalyzer()
    sentiment_scores = sia.polarity_scores(cleaned_text)

    ## Returns the length-four sentiment dictionary in any order
    return sentiment_scores

In [381]:
## Running it on a few examples:
func("Barack Obama spoke at the United Nations in 2014. He said the U.S. economy had improved.")
func("The U.S. News and Report (a special website) named Dartmouth College as one of the top universities. This made all of its undergraduates extremely happy, so they celebrated. People who don't go to Dartmouth College were very angry about this and called them up.")


{'neg': 0.0, 'neu': 0.744, 'pos': 0.256, 'compound': 0.4767}

{'neg': 0.078, 'neu': 0.651, 'pos': 0.271, 'compound': 0.835}

In [383]:
## your code here executing the function:
doj_sentiment_scores = [func(press_rel) for press_rel in doj_subset["contents"]]

#check:
doj_sentiment_scores

[{'neg': 0.199, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931},
 {'neg': 0.133, 'neu': 0.799, 'pos': 0.068, 'compound': -0.9325},
 {'neg': 0.093, 'neu': 0.83, 'pos': 0.077, 'compound': -0.7579},
 {'neg': 0.126, 'neu': 0.789, 'pos': 0.085, 'compound': -0.9037},
 {'neg': 0.178, 'neu': 0.778, 'pos': 0.044, 'compound': -0.9864},
 {'neg': 0.146, 'neu': 0.802, 'pos': 0.052, 'compound': -0.987},
 {'neg': 0.16, 'neu': 0.762, 'pos': 0.078, 'compound': -0.9595},
 {'neg': 0.092, 'neu': 0.843, 'pos': 0.065, 'compound': -0.7783},
 {'neg': 0.105, 'neu': 0.834, 'pos': 0.061, 'compound': -0.9136},
 {'neg': 0.166, 'neu': 0.778, 'pos': 0.056, 'compound': -0.9801},
 {'neg': 0.213, 'neu': 0.745, 'pos': 0.042, 'compound': -0.9972},
 {'neg': 0.108, 'neu': 0.827, 'pos': 0.065, 'compound': -0.9153},
 {'neg': 0.082, 'neu': 0.853, 'pos': 0.065, 'compound': -0.6486},
 {'neg': 0.323, 'neu': 0.638, 'pos': 0.039, 'compound': -0.995},
 {'neg': 0.181, 'neu': 0.76, 'pos': 0.059, 'compound': -0.9907},
 {'neg': 0.124,

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [385]:
## PART C:
doj_subset_wscore = doj_subset.copy()

doj_subset_wscore["sentiment_scores"] = doj_sentiment_scores

doj_subset_wscore["neg"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["neg"])
doj_subset_wscore["neu"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["neu"])
doj_subset_wscore["pos"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["pos"])
doj_subset_wscore["compound"] = doj_subset_wscore["sentiment_scores"].apply(lambda x: x["compound"])

highest_neg_scores = doj_subset_wscore.sort_values(by="neg", ascending=False)

#print(highest_neg_scores[["id", "contents", "neg"]].head(2))
# also without print, since the formatting is kinda ugly:

highest_neg_scores[["id", "contents", "neg"]].head(2)


,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.323
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.303


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [387]:
## Part D:
mean_sentiment_by_topic = doj_subset_wscore.groupby("topics_clean")["compound"].agg("mean")
mean_sentiment_by_topic


topics_clean
Civil Rights             -0.093931
Hate Crimes              -0.930943
Project Safe Childhood   -0.681391
Name: compound, dtype: float64

**Part E:**
We likely observe this variation in scores because different emotional tones and public perceptions are associated with each issue; for instance, Hate Crimes and Project Safe Childhood may have more negative sentiments due to their often tragic nature while Civil Rights may be viewed more neutrally, considering there are positive contexts in which Civil Rights-related words are discussed.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [389]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [391]:

## PART A:

def preprocess(string):
    
    ## Convert word to lowercase:
    str_lower = string.lower()

    ## Remove stopwords, adding custom stopwords to default list:
    stopword_list = stopwords.words("english")
    new_stopwords_list = stopword_list + custom_doj_stopwords
    str_token = [word for word in wordpunct_tokenize(str_lower) if word not in new_stopwords_list]

    ## Only retains alpha words and words with more than 3 characters using the snowball stemmer from nltk to stem:
    stemmer = SnowballStemmer("english")
    str_stem = [stemmer.stem(tok) for tok in str_token if tok.isalpha() and len(tok) > 3]

    ## Return a joined preprocessed string:
    return " ".join(str_stem)


In [393]:
## your code executing the function
## PART B:
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(preprocess)


## checks
# doj_subset_wscore.head()
# doj_subset_wscore[["processed_text", "contents"]]


In [395]:
## your code showing the examples
## PART C:
id_list = ["16-718", "16-217"]
example = doj_subset_wscore[doj_subset_wscore["id"].isin(id_list)][["id", "contents", "processed_text"]]
example

,id,contents,processed_text
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv offic involv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem offic involv shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc offic involv shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim offic involv shoot effect quick investig offic involv shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc agreement today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr consti

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [397]:
def create_dtm(list_of_strings, metadata):
    
    vectorizer = CountVectorizer(lowercase = True)
    
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    
    return(dtm_dense_named_withid)

In [399]:
# PART A: 
strings_list = doj_subset_wscore["processed_text"]
updated_doj = doj_subset_wscore.rename(columns={"compound": "compound_sent"})
#to avoid duplicated "compound" or "id" columns once we create the dtm

data = updated_doj[["id", "compound_sent", "topics_clean"]]

dtm = create_dtm(strings_list, data)

## check
#dtm

In [401]:
## PARTS B-D:

## creating all subset data frames
bcutoff= dtm["compound_sent"].quantile(0.95)
B_sub = dtm[dtm["compound_sent"] > bcutoff].reset_index()


ccutoff= dtm["compound_sent"].quantile(0.05)
C_sub = dtm[dtm["compound_sent"] < ccutoff].reset_index(drop = True)

D_cr = dtm[dtm["topics_clean"] == "Civil Rights"]
D_hc = dtm[dtm["topics_clean"] == "Hate Crimes"]
D_psc = dtm[dtm["topics_clean"] == "Project Safe Childhood"]


## function to find topwords for the different subsets
def get_topwords(df_sub):
    word_columns = df_sub.drop(columns=["index", "id", "compound_sent", "topics_clean"])
    word_sums = word_columns.sum().sort_values(ascending=False)
    print(word_sums.head(10))
    print("\n")



get_topwords(B_sub)
get_topwords(C_sub)
get_topwords(D_cr)
get_topwords(D_hc)
get_topwords(D_psc)



level_0       10579
agreement       171
enforc          130
state           110
disabl          107
ensur           107
communiti        99
student          87
settlement       86
servic           83
dtype: int64


assault     191
crime       171
victim      164
hate        129
defend      126
offic       119
conspir     105
sentenc     104
charg        97
american     95
dtype: int64


offic        637
hous         633
discrimin    616
enforc       544
disabl       532
said         497
feder        479
violat       477
state        452
court        414
dtype: int64


victim      591
crime       557
hate        524
defend      484
prosecut    478
charg       463
sentenc     455
american    451
feder       432
guilti      430
dtype: int64


child          1022
exploit         701
sexual          572
safe            479
childhood       474
project         472
pornographi     452
children        423
crimin          405
prosecut        374
dtype: int64




## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [403]:
# your code here (drawing from class on 5/6)
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

## Step 1: re-tokenize and store in list
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  doj_subset_wscore.processed_text]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below


## Step 3: apply dictionary to TOKENIZED texts

corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]


## Step 4: we're finally ready to estimate the model!

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict,
                                         random_state = 100,
                                         passes=15, 
                                         alpha = 'auto',
                                         per_word_topics = True)

print(type(ldamod))


<class 'gensim.models.ldamodel.LdaModel'>


In [404]:
## Post-model 1: 

topics = ldamod.print_topics(num_words = 15)

for topic_num, topic_words in topics:
    print(f"Topic: {topic_num +1}")
    print(" -" + topic_words.replace(" + ", "\n -"))
    print("\n")


## Display
lda_display = gensimvis.prepare(ldamod, corpus_fromdict, text_raw_dict)
pyLDAvis.display(lda_display)

Topic: 1
 -0.019*"child"
 -0.013*"exploit"
 -0.011*"sexual"
 -0.010*"sentenc"
 -0.010*"prosecut"
 -0.010*"crimin"
 -0.009*"offic"
 -0.009*"victim"
 -0.009*"safe"
 -0.009*"childhood"
 -0.008*"project"
 -0.008*"pornographi"
 -0.008*"year"
 -0.008*"children"
 -0.008*"investig"


Topic: 2
 -0.011*"hous"
 -0.011*"discrimin"
 -0.009*"disabl"
 -0.008*"enforc"
 -0.007*"agreement"
 -0.007*"state"
 -0.007*"said"
 -0.006*"court"
 -0.006*"violat"
 -0.006*"alleg"
 -0.006*"feder"
 -0.005*"requir"
 -0.005*"general"
 -0.005*"settlement"
 -0.005*"fair"


Topic: 3
 -0.011*"victim"
 -0.011*"crime"
 -0.011*"defend"
 -0.010*"charg"
 -0.010*"hate"
 -0.009*"prosecut"
 -0.009*"said"
 -0.009*"sentenc"
 -0.008*"american"
 -0.008*"feder"
 -0.008*"guilti"
 -0.008*"indict"
 -0.008*"assault"
 -0.007*"african"
 -0.007*"year"




## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [425]:
## your code here to add those topic probabilities to the dataframe

## Part A:
topic_probs = [ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
len(topic_probs) == len(doj_subset_wscore)



## Part B:
# Convert topic_probs into a DataFrame
topic_df = pd.DataFrame([[prob for i, prob in doc] for doc in topic_probs],
                        columns=["Topic_1", "Topic_2", "Topic_3"])

# Join topic probabilities to main DataFrame
doj_subset_wscore = doj_subset_wscore.reset_index(drop=True).join(topic_df)

# Add column for most probable topic
doj_subset_wscore["top_topic"] = topic_df.idxmax(axis=1)


doj_subset_wscore

True

ValueError: columns overlap but no suffix specified: Index(['Topic_1', 'Topic_2', 'Topic_3'], dtype='object')

In [427]:
## check
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,sentiment_scores,neg,neu,pos,compound,processed_text,Topic_1,Topic_2,Topic_3,top_topic
0,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle","{'neg': 0.199, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}",0.199,0.751,0.049,-0.9931,former supervisori correct offic louisiana state penitentiari angola louisiana plead guilti yesterday connect beat handcuf shackl inmat addit conspir cover misconduct falsifi offici record lie intern investig happen jame savoy marksvill louisiana admit plea hear wit offic use excess forc inmat fail interven conspir offic cover beat engag varieti obstruct act person falsifi offici prison record cover attack scotti kennedi beeb arkansa john sander marksvill louisiana previous plead guilti novemb septemb role beat cover everi citizen right process protect unreason forc correct offic violat basic constitut must held account egregi action said act general john gore continu vigor prosecut correct offic violat public trust commit crime cover violat feder crimin yesterday anoth exampl unwav commit pursu violat feder crimin law said act unit state middl louisiana corey amundson continu work close ensur investig baton roug resid agenc prosecut frederick menner middl louisiana christoph perra crimin section,0.477705,0.133606,0.388689,Topic_1
1,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a D

In [429]:
## your code here to summarize the topic proportions for each of the topics_clean 

## Part C
#For each of the manual labels in topics_clean (Hate Crime, Civil Rights, Project Safe Childhood), 
#print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- 
#if 123 of those documents are coded to topic_1, that would be 50%; and so on). Hint: pd.crosstab and normalize may be helpful: 
#https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

topic_distribution = pd.crosstab(index = doj_subset_wscore["topics_clean"], columns = doj_subset_wscore["top_topic"], normalize = "index")

# multiply everything by 100 to get percentages
topic_distribution = topic_distribution.mul(100)

print(topic_distribution)


top_topic                  Topic_1    Topic_2    Topic_3
topics_clean                                            
Civil Rights             10.163934  69.508197  20.327869
Hate Crimes               9.349593   0.000000  90.650407
Project Safe Childhood  100.000000   0.000000   0.000000


In [ ]:
## Part D-->explanation of the probability distribution

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [480]:
#Part A
def create_bigram_onedoc(proc_text_string):

    words = proc_text_string.split()
    bigrams = [f"{words[i]}_{words[i+1]}" for i in range(len(words)-1)]
    return " ".join(bigrams) #to return it as a string that is the paur of words with the birgrams structured with underscores

doj_subset_wscore["processed_text_bigrams"] = doj_subset_wscore["processed_text"].apply(create_bigram_onedoc)

## check
#doj_subset_wscore.head()

## Part B
example = doj_subset_wscore[doj_subset_wscore["id"] == "16-217"]
#example[["id", "processed_text", "processed_text_bigrams"]]

## Part C:

strings_list_bigrams = doj_subset_wscore["processed_text_bigrams"]
updated_doj = doj_subset_wscore.rename(columns={"compound": "compound_sent"})
#to avoid duplicated "compound" or "id" columns once we create the dtm

data_bigram = updated_doj[["id", "topics_clean", "compound_sent"]]

dtm_bigram = create_dtm(strings_list_bigrams, data)

## check
#dtm_bigram
dtm_bigram.shape
dtm.shape

(717, 72721)

(717, 6870)

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [ ]:
# your code here 